# Proyecto 2

* Flavio Galán
* Josue Say
* Juan Pablo Solís
* Isabella Miralles

[Link Kaggle](https://www.kaggle.com/competitions/lmsys-chatbot-arena)
## Dependencias

In [2]:
import pandas as pd

## Situación Problemática

## Problema Científico

## Objetivos

## Análisis Exploratorio

### Técnicas comunes para detección de patrones en texto


### Preprocesamiento de los datos

Primero tenemos que cargar el dataset, Kaggle ya nos divide los datos en entrenamiento y prueba por lo que tenemos:

In [13]:
df = pd.read_csv("data/train.csv")
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [43]:
ap = df["response_a"].iloc[0]
print(ap)
ap = ap.replace("\\\\", "\\")
print(ap)
a = eval(ap)
"\n".join(a)

["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers.\n\n4. **Role Modeling:

'The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in management can help ensure that women have equal opportunities to advance in their careers.\n\n4. **Role Modeling:*

Por lo que podemos ver que el dataset contiene el prompt dado a la LLM y la respuesta que dieron. En el dataset de entrenamiento se incluye tanto el modelo que respondió la pregunta como el ganador o si hubo un empato, sin embargo en el dataset de prueba solamente se tiene la respuesta y el prompt, por lo que no podremos usar las variables modelo de forma directa para entrenar a nuestro modelo.

### Descripción de las variables

* **id**: ID de la interacción entre un usuario y los modelos.
* **model_a**: El nombre del modelo identificado como A en el enfrentamiento. No se puede usar para entrenar el dataset.
* **model_b**: El nombre del modelo identificado como B en el enfrentamiento. No se puede usar para entrenar el dataset.
* **prompt**: Un array de cadenas de texto que representan los inputs dados al modelo.
* **response_a**: La respuesta que dió el LLM identificado con a.
* **response_b**: La respuesta qeu dió el LLM identificado con b.
* **winner_model_a**: Si el modelo a fue elegido como ganador.
* **winner_model_b**: Si el modelo b fue elegido como ganador.
* **winner_tie**: Si hubo un empate entre los modelos.

Para facilitar el análisis de los datos se parseó cada columna del dataframe de prompt, response[ab] y se unieron en una sola cadena de texto. Tenemos entonces:

In [50]:
def eval_and_join(row, key: str):
    try:
        p_code = row[key]
        print(f"Parsing: {p_code}")
        arr = eval(p_code)
        return "\n".join(arr)
    except:
        print("Failed to parse!")
        #print(f"Failed to parse: {p_code}")

def parse_array_and_join(key: str):
    return lambda row: eval_and_join(row, key)

df["prompt_clean"] = df.apply(parse_array_and_join("prompt"), axis=1)
# df["response_a_clean"] = df.apply(parse_array_and_join("response_a"), axis=1)
# df["response_b_clean"] = df.apply(parse_array_and_join("response_b"), axis=1)
df

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,prompt_clean,response_a_clean,response_b_clean
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi..."
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...
...,...,...,...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0,"A simple mnemonic for π:\n""How I wish I could ...","Sure, let's break it down:\n\n1. ""How"" has 3 l...",Here is how that mnemonic represents the digit...
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0,"In python, implement a naive Bayes with gaussi...",Here is an implementation of a naive Bayes cla...,Sure! Here's an implementation of a naive Baye...
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0,is it unethical to work on building weapons?\n...,Working on weapons technology raises some ethi...,It depends on the context. Weapons can be used...
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0,"If a bait contains 0,0025% bromadiolon then ho...",Bromadiolone is a rodenticide which is most of...,"As an AI language model, I do not promote or c..."


### EDA

## Conclusiones